In [99]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
import json

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

### Selenium

In [3]:
from selenium import webdriver

def init():
    chrome_driver = 'chromedriver' # 드라이버 위치 확인
    driver = webdriver.Chrome(chrome_driver) # 드리이버 만들기
    return driver

In [9]:
cookiedict = {
    'gnbFav' : '%7B%22gnbFav%22%3A%5B%5D%7D',
    'NID_SAUTO' : '-1891493465',
    'coach_tooltip' : 'ok',
    'nid_enctp' : '1',
    'nid_buk' : '43O5YWJ7SNCV2',
    'NID_SES' : 'AAABeAgr0Glx2yCbCp+qjOuI+ro1F0g9h8GN/OVScdu3XzFA/o8Sosg8tQm7RsgV3xLkUpmoP9c3eB2Z2BECpqHvQ69V0Pm1uv/DceHH0Iki/99mWxwh12PTo/zWu8IES6aidySdoo2IJgtddR1S8aodlBDbZgrlQZhXWov3gzb/JVAeu9B5L+Ra6AXsKfyp3yKV9b4pzsleg3TLZ4Eo4YXOv0pb1rEWcZzzHRXjE68Tz47n6za9gy8RKLjLOXQ7nKCOZa00j6d0/i8wdIAun3EkUK0prRQGkaEczzrfDH/2ItilduqDcBeEnitmfdve+6dfJ4nFJb8hQsuvAnRtJQC8ewnayL0c6oBdNCccLGL5yU0IKRK8FdxrgfUDT0t5sChiW9MWekCRf+JRz5JIhOmlX91/t8iUWCMnKxjYTCbiXmW6OethS5E6LP+2SX7Aajnsj0+XfapIGTzpmA/ZjkD1WRm7t6jwU22F6KF9dCsFp2gZbE1eIrKUVIETtD8HLY1IKg==',
    'NMUSER' : 'YXKmFAEwaqbsaAUqFxKdbXk5W40N1HtlFxgXaqu9axEqaAtZaxusFxuZKAnmaqns3xndFAKwFoUsHoKma9vsxonOaxRVadUstoRVaqRVaqns1rejL9Us6xRVaqnD16lvpB2RKZl5WLl5MBp0bSloWrdnaAvmKARqp6FTW43CbNvR16lvpB2RKZl5WLl5MBp0bSloWrdnaAvmKAn=',
    'NID_AUT' : 'Uu3mELeSQvuRqLKCvunNeRM/eLeoakeJZgc0tHYAtWpos96czTnjcTs+Daguji1r',
    '_ga_7VKFYR6RV1' : 'GS1.1.1595347015.33.1.1595348957.60',
    'NID_JKL' : 'ETkTt2IWJyid9+q8wvnujZTI2M9fOFyIkTtSB1LgmFc=',
    '_ga' : 'GA1.1.2123753588.1591027971',
    'nid_slevel' : '-1',
    '_gid' : 'GA1.2.283987012.1595347016',
    'NRTK' : 'ag#all_gr#4_ma#2_si#2_en#2_sp#2',
    'nid_inf' : '-600244553',
    'nx_ssl' : '2',
    'ASID' : 'b6dd6ef1000001721be570c10000005f',
    'NNB' : 'VGJQ2CTCLK7V4',
}

### 네이버 스팸 메일

In [58]:
driver = webdriver.Chrome()

# driver.delete_all_cookies() # 모든 쿠키 삭제
# driver.get_cookies() # 쿠키 확인
driver.get('http://www.naver.com')

# Login 되었을 때의 쿠기 넣어주기
for k,v in coodiedict.items():
    driver.add_cookie({'name' : k, 'value' : v, 'domain' : '.naver.com'})
    
driver.get('http://www.naver.com')

driver.get('http://mail.naver.com')

# spam mail menu click
# menu = driver.find_element_by_css_selector('div.list_menu')[-1]
# menu.find_element_by_css_selector('a[href]').click()

# spam mail 가져오기
# for _ in driver.find_element_by_xpath('//ol//li//div[#class="mTitle"]'):
#     print(_.text)

# driver.close()

### NAVER Log Out

In [122]:
from selenium.common.exceptions import ElementNotVisibleException, ElementNotInteractableException, TimeoutException

In [123]:
driver = webdriver.Chrome()

# driver.delete_all_cookies() # 모든 쿠키 삭제
# driver.get_cookies() # 쿠키 확인
driver.get('http://www.naver.com')

# Login 되었을 때의 쿠기 넣어주기
for k,v in coodiedict.items():
    driver.add_cookie({'name' : k, 'value' : v, 'domain' : '.naver.com'})
    
driver.get('http://www.naver.com')

driver.get('http://mail.naver.com')

In [124]:
btn = driver.find_element_by_id('gnb_logout_button')
btn.is_displayed(), btn.is_enabled()

(False, True)

In [125]:
btn = driver.find_element_by_id('gnb_logout_button')
btn.is_displayed(), btn.is_enabled()

(False, True)

In [127]:
from selenium.webdriver.support.ui import WebDriverWait

# WebDriverWait() # driver wait 시키기

wait = WebDriverWait(driver, 5, 0.5,
                     (ElementNotVisibleException,ElementNotInteractableException,TimeoutException))

In [128]:
try:
    wait.until(lambda d:d.find_element_by_id('gnb_logout_button').is_displayed())
    print(btn.is_displayed(), btn.is_enabled())
except (ElementNotVisibleException,
       ElementNotInteractableException,
       TimeoutException) as e:
    driver.find_element_by_css_selector('#gnb_my_namebox > a').click()
    print(btn.is_displayed(), btn.is_enabled())
finally:
    pass # 이후 작업

True True


### 예제 사이트

In [44]:
driver = webdriver.Chrome()
driver.get('http://example.webscraping.com/places/default/search')
driver.find_element_by_xpath('//input[@id="search_term"]').clear()
driver.find_element_by_xpath('//input[@id="search_term"]').send_keys('korea')
driver.find_element_by_xpath('//input[@id="search"]').click()

In [45]:
for _ in driver.find_elements_by_xpath('//div[@id="results"]//td/div'):
    print(_.find_element_by_css_selector('a').get_attribute('href'))
    print(_.find_element_by_css_selector('img').get_attribute('src'))
    print(_.find_element_by_css_selector('a').text.strip())
    print()

### 논문 사이트

In [56]:
driver = webdriver.Chrome()
driver.get('https://pubmed.ncbi.nlm.nih.gov')
driver.find_element_by_id('id_term').clear()
driver.find_element_by_id('id_term').send_keys('COVID-19')
driver.find_element_by_class_name('search-btn').click()

for _ in range(2):
    sleep(1)
    driver.find_element_by_css_selector('.results-amount + button').click()

for _ in driver.find_elements_by_css_selector('a.docsum-title'):
    print(_.text)

The neuroinvasive potential of SARS-CoV2 may play a role in the respiratory failure of COVID-19 patients.
[Strategies for vaccine development of COVID-19].
COVID-19 (Novel Coronavirus 2019) - recent trends.
COVID-19 epidemic: Disease characteristics in children.
Understanding of COVID-19 based on current evidence.
Unique epidemiological and clinical features of the emerging 2019 novel coronavirus pneumonia (COVID-19) implicate special control measures.
Perspectives on monoclonal antibody therapy as potential therapeutic intervention for Coronavirus disease-19 (COVID-19).
What dentists need to know about COVID-19.
Outcomes in Patients with COVID-19 Infection Taking ACEI/ARB.
Current Status of Epidemiology, Diagnosis, Therapeutics, and Vaccines for Novel Coronavirus Disease 2019 (COVID-19).
Review of the Clinical Characteristics of Coronavirus Disease 2019 (COVID-19).
COVID-19: a review.
[Analysis of the pregnancy outcomes in pregnant women with COVID-19 in Hubei Province].
Coronavirus d

### YouTube

ElementNotInteractableException => 진짜로 눈에 안보일때(있지만 가려진 경우)
ElementNotVisibleException => 코드에는 있으나 화면에 없을 때

In [93]:
driver = webdriver.Chrome()
query = '인디'
driver.get('https://www.youtube.com/results?search_query=' + query)

video = []
for _ in driver.find_elements_by_css_selector('ytd-video-renderer'):
#     video = _.find_element_by_css_selector('a')
#     desc = _.find_element_by_css_selector('a#video-title')
#     print(video.get_attribute('href'))
#     print(desc.get_attribute('aria-label'))
    
    video.append(_.find_element_by_css_selector('a').get_attribute('href'))

In [94]:
driver.get(video[0])

In [95]:
dom = BeautifulSoup(driver.page_source, 'html.parser')

In [97]:
dom.select_one('video')['src']

'blob:https://www.youtube.com/e47eb069-1848-4d5a-8c34-b6798ce96245'

In [103]:
from urllib.request import urlopen, Request

req = Request(dom.select_one('video')['src'], headers = headers)
resp = urlopen(req)

In [109]:
resp = request('GET', 'https://r5---sn-ab02a0nfpgxapox-bh2ls.googlevideo.com/videoplayback?expire=1595495410&ei=kv8YX5rVFYOPlQTZ6a-AAw&ip=182.221.110.241&id=o-AC2im0XDCppp3OAI8Lz_M88E5rczwizyBVZMRCPCio9T&itag=251&source=youtube&requiressl=yes&mh=Jp&mm=31%2C29&mn=sn-ab02a0nfpgxapox-bh2ls%2Csn-ab02a0nfpgxapox-jwwe7&ms=au%2Crdu&mv=m&mvi=5&pl=26&usequic=no&initcwndbps=2577500&vprv=1&mime=audio%2Fwebm&gir=yes&clen=55442009&dur=3148.581&lmt=1587958430412590&mt=1595473648&fvip=5&keepalive=yes&fexp=23883098&c=WEB&txp=5311222&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cusequic%2Cinitcwndbps&lsig=AG3C_xAwRQIhAJLZMJe1pM4j2YfzhnirAnJmP9543p_Bn8dAb6hWLIJ3AiBNYSYw533sfrBHLYGQgg0-uuF1q-EFWmcC3Yk0eXUluA%3D%3D&alr=yes&sig=AOq0QJ8wRgIhAKOmrD9THzHi7OBRPfXeOv3BRlC8h4lYBF8ikZF0PHGQAiEAnub3jednySxjkVBHRXUO7Y-CpuyJDLY6f1TcCq_yO9c%3D&cpn=fjlJp2bIH-mbs6FB&cver=2.20200721.01.02&range=1999415-2519444&rn=11&rbuf=99088')

In [112]:
url = 'https://r5---sn-ab02a0nfpgxapox-bh2ls.googlevideo.com/videoplayback?expire=1595495410&ei=kv8YX5rVFYOPlQTZ6a-AAw&ip=182.221.110.241&id=o-AC2im0XDCppp3OAI8Lz_M88E5rczwizyBVZMRCPCio9T&itag=397&aitags=133%2C134%2C135%2C136%2C137%2C160%2C242%2C243%2C244%2C247%2C248%2C278%2C394%2C395%2C396%2C397%2C398%2C399&source=youtube&requiressl=yes&mh=Jp&mm=31%2C29&mn=sn-ab02a0nfpgxapox-bh2ls%2Csn-ab02a0nfpgxapox-jwwe7&ms=au%2Crdu&mv=m&mvi=5&pl=26&usequic=no&initcwndbps=2577500&vprv=1&mime=video%2Fmp4&gir=yes&clen=23807536&dur=3148.545&lmt=1588138394585759&mt=1595473648&fvip=5&keepalive=yes&fexp=23883098&c=WEB&txp=5311222&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cusequic%2Cinitcwndbps&lsig=AG3C_xAwQwIgfqZh3mCgMzG8qWpT779VmulSqzcvwR-NO3oUgF0-EMsCH1NiFFFCwg8E-5QoYhV1yIcDl9HeNF2mfncJCZBJDzo%3D&alr=yes&sig=AOq0QJ8wRAIgKPdKr5T8xP_rCYMQZ50nqeUXpd81SDXUpjIigzmjxS4CIBbfSz1YCWKeBsuiaVxPobo2wB5cU7WdDdbquxA032mA&cpn=fjlJp2bIH-mbs6FB&cver=2.20200721.01.02&rn=12&rbuf=119674&range='
with open('test.mp4', 'wb') as f:
    for _ in range(100):
        resp = request('GET',
                       url = url + '-'.join([str(_*100000), str((_+1)*100000)]),
                      headers=headers)
        f.write(resp.content)

### Preprocessing

- 형태소 분석
    - 토큰분리, 어간 추출, 품사 부착, 색인, 벡터화
- 구문 분석
    - 문장 경계 인식, 구문분석, 공기어, 개체명 사전 구축, 개채명 인식(PLOT, 수치, 외국어 한글 표기)
- 의미 분석
    - 대용어 해소(대명사, 두문자어, 약어, 수치), 의미 중의성 해결(동명이인)
- 담론 분석
    - 분류, 군집, 중복, 요약, 가중치, 순위화, 토픽 모델링, 이슈 트래킹, 평판분석, 감성분석, 복합논증분석

In [131]:
from konlpy.tag import Kkma
Kkma().pos('설치여부 확인법')

[('설치', 'NNG'), ('여부', 'NNG'), ('확인', 'NNG'), ('벌', 'VV')]

In [132]:
from nltk.tag import pos_tag
pos_tag('you need python.'.split(' '))

[('you', 'PRP'), ('need', 'VBP'), ('python.', 'NNS')]

In [133]:
# nltk.download('gutenberg', 'stopwords', 'punkt')

In [145]:
from nltk.corpus import gutenberg

# print(gutenberg.open(gutenberg.fileids()[0]).read())
corpus = gutenberg.open(gutenberg.fileids()[0]).read()

len(corpus.splitlines()), len(corpus.split())

(16823, 158167)

In [146]:
# Tokenizing => 비정형데이터를 Token 단위로 나누는 방법
# 1. Sentence Tokenizing
# 2. Word(어절) Tokenizing
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import regexp_tokenize

In [151]:
(len(sent_tokenize(corpus)), 
len(word_tokenize(corpus)), 
len(TweetTokenizer().tokenize(corpus)),
len(regexp_tokenize(corpus, r'\b(\w+)\b')))

(7493, 191785, 193228, 161983)

In [167]:
data = '''
블랙핑크 멤버 4인 컷 아래 남겨진 물음표는 
음악 팬들의 궁금증을 더욱 자극했다. 
지난 5월 세계적인 팝스타 레이디 가가와의 협업곡으로 
글로벌 음악 시장을 뒤흔들었던
블랙핑크의 파격적인 콜라보레이션을 짐작할 수 있는 대목.
'''

In [171]:
# \n 기준
data.splitlines()

# 문장부호 문장을 분리
# ., !, ? => 분리
# 문장 중간에 나오면 분리하지 않음
# 문어체 -> 가능
sent_tokenize(data)

['\n블랙핑크 멤버 4인 컷 아래 남겨진 물음표는 \n음악 팬들의 궁금증을 더욱 자극했다.',
 '지난 5월 세계적인 팝스타 레이디 가가와의 협업곡으로 \n글로벌 음악 시장을 뒤흔들었던\n블랙핑크의 파격적인 콜라보레이션을 짐작할 수 있는 대목.']

In [186]:
print(len(word_tokenize(data))) # 동양 ex) ^^ O / :) X
print(len(TweetTokenizer().tokenize(data))) # 서양 ex) ^^ X / :) O
print(len(regexp_tokenize(data, r'\b(\w+)\b')))

32
32
30


In [187]:
from nltk import Text

emma = Text(word_tokenize(corpus))

In [194]:
len(emma), len(set(emma)) # 고유한 단어 갯수

(191785, 8406)

In [195]:
emma.vocab().most_common(10)

[(',', 12016),
 ('.', 6355),
 ('to', 5125),
 ('the', 4844),
 ('and', 4653),
 ('of', 4272),
 ('I', 3177),
 ('--', 3100),
 ('a', 3001),
 ("''", 2452)]